In [ ]:
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error,mean_squared_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from tqdm import tqdm

In [ ]:
import sys
sys.path.append('temp/temp/')
import make_tissue as mt

In [ ]:
def mean_OD(spectra,wvs,wvl,window):
    left_wvl = wvl-(window/2.)
    right_wvl = wvl+(window/2.)
    wvl_mask = (left_wvl<=wvs) & (wvs<=right_wvl)
    return spectra[wvl_mask].mean()
def mean_R(spectra,wvl,window):
    left_wvl = wvl-(window/2.)
    right_wvl = wvl+(window/2.)
    wvl_mask = (left_wvl<=all_wvs) & (all_wvs<=right_wvl)
    return spectra[wvl_mask].mean()
def line(spectra,wvl_1,wvl_2):
    y_1 = mean_OD(spectra,wvl_1,2.)
    y_2 = mean_OD(spectra,wvl_2,2.) 
    a = (y_2 - y_1)/(wvl_2-wvl_1)
    b = (y_1*wvl_2-y_2*wvl_1)/(wvl_2-wvl_1)
    return a*wvs+b
def line_correction(spectra,left_wvl,right_wvl):
    return spectra - line(spectra,left_wvl,right_wvl)
def norm(y):
    return (y-y.min())/(y.max()-y.min())

## Загрузка моделей

In [ ]:
knn_paths = sorted(glob.glob('*.joblib'))

In [ ]:

csv_file='2_layered_model_mcml.csv'

data = pd.read_csv(csv_file)

distances = np.arange(0.0025,1.5,0.005).round(4)

data.columns = np.append(distances,['mua1', 'mus1', 'd1','mua2', 'mus2','d2'])
data.drop((np.arange(0.9975,1.5,0.005).round(4).astype(str)), axis=1, inplace=True)
data.drop('d2', axis=1, inplace=True)
optical = ['mua1','mus1','d1','mua2','mus2']
Rs = [r for r in data.columns if r not in optical]
dr=0.005
for col in Rs:
    distance = float(col)
    data[col] = 2*np.pi*distance*dr*data[col]

mu_coefs = ['mua1','mus1','mua2','mus2']

for coef in mu_coefs:
    log_name = f'log_{coef}'
    data[log_name]=np.log(data[coef])
    data[log_name]=(data[log_name]-data[log_name].min())/(data[log_name].max()-data[log_name].min())


In [ ]:

log_and_norm_optical = ['log_mua1', 'log_mus1',
       'log_mua2', 'log_mus2','mua1','mus1','mua2','mus2','d1']


In [ ]:

optical_prop = data[log_and_norm_optical]
data_R = data[Rs]
X_train, X_test, ys_train, ys_test = train_test_split(optical_prop, data_R, test_size=0.10, random_state=42)

## Моделирование спектров

In [ ]:
wvs = np.linspace(700,1100,200)

In [ ]:
mt.make_tissue_list(800)

In [ ]:
second_tissue = []
mua2 = []
mus2 = []
for wv in wvs:
    _mua,_mus,_ = mt._get_standard_tissue(wv,
                                          B=0.004,S=0.5,W=0.2,M=0.,F=.8,
                                      musp500=22,fray=0.,bmie=0.68,gg=0.9)
    mua2.append(_mua)
    mus2.append(_mus)
second_tissue.append({'mua2':mua2,'mus2':mus2})

In [ ]:
plt.plot(wvs,second_tissue[0]['mua2'])

In [ ]:
water_concs = np.linspace(0.50,0.95,10)

In [ ]:
water_concs

In [ ]:
first_tissue = []
mua2 = []
mus2 = []
for wv in wvs:
    _mua,_mus,_ = mt._get_standard_tissue(wv,
                                          B=0.004,S=0.67,W=0.65,M=0.,F=0.00,musp500=48.0,fray=0.41,bmie=0.562,gg=0.9)
    mua2.append(_mua)
    mus2.append(_mus)
first_tissue.append({'mua1':mua2,'mus1':mus2})

In [ ]:

plt.plot(wvs,first_tissue[0]['mua1'],label='dermis')
plt.plot(wvs,second_tissue[0]['mua2'],label='subcutaneous fat')
plt.legend()
plt.xlabel('Wavelength, nm')
plt.ylabel('$\mu_a$, $cm^{-1}$')

In [ ]:
#for tissue in first_tissues[::10]:
plt.plot(wvs,first_tissue[0]['mus1'],label='dermis')
plt.plot(wvs,second_tissue[0]['mus2'],label='subcutaneous fat')
#plt.axhline(100)
plt.legend()
plt.xlabel('Wavelength, nm')
plt.ylabel('$\mu_s$, $cm^{-1}$')

In [ ]:
def norm_mu(mua,coef='mua1'):
    mua = np.log(mua)
    
    return (mua-np.log(data[coef]).min())/(np.log(data[coef]).max()-np.log(data[coef]).min())

In [ ]:
models_paths_dict = {}
models_dict = {}
for path in knn_paths:
    d1 = path.split('_')[-1].split('joblib')[0]
    models_paths_dict[d1]=[]
    models_dict[d1]=[]

In [ ]:
for key in models_paths_dict.keys():
    for path in knn_paths:
        if key in path:
            models_paths_dict[key].append(path)

In [ ]:
for key in models_paths_dict.keys():
    for path in models_paths_dict[key]:
        models_dict[key].append(load(path))

In [ ]:
models_dict

In [ ]:
for d1 in sorted([0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]):   
    plt.figure()
    wvs_mask= (wvs>700)
    plt.title('dermis thickness = {} cm'.format(d1))
    i=0
    for model,d in zip(models_dict[str(d1)][::20],Rs[::20]): 
        spectra_layer =  np.array([norm_mu(first_tissue[0]['mua1'],'mua1'),
                       norm_mu(first_tissue[0]['mus1'],'mus1'),
                       norm_mu(second_tissue[0]['mua2'],'mua2'),
                       norm_mu(second_tissue[0]['mus2'],'mus2')]).T
        R_spectra=model.predict(spectra_layer)
        y = -np.log(R_spectra.T)[wvs_mask]
        x = wvs[wvs_mask]
        plt.plot(x,y-mean_OD(y,x,850,5),'-',label=f'{d} cm')
        i+=1
    plt.xlabel('Wavelength, nm')
    plt.ylabel('OD')
    plt.legend(title='fiber distance',loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
ratio = {}
lipids = {}
water = {}
for d1 in sorted([0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]):   
    wvs_mask= (wvs>700)
    ratio[d1] = []
    lipids[d1] = []
    water[d1]=[]
    i=0
    for model,d in zip(models_dict[str(d1)+'.'][::],Rs[::]): 
        spectra_layer =  np.array([norm_mu(first_tissue[0]['mua1'],'mua1'),
                       norm_mu(first_tissue[0]['mus1'],'mus1'),
                       norm_mu(second_tissue[0]['mua2'],'mua2'),
                       norm_mu(second_tissue[0]['mus2'],'mus2')]).T
        R_spectra=model.predict(spectra_layer)
        y = -np.log(R_spectra.T)[wvs_mask]
        x = wvs[wvs_mask]
        i+=1
        ratio[d1].append((mean_OD(y,x,930,5)-mean_OD(y,x,850,5))/((mean_OD(y,x,970,5)-mean_OD(y,x,850,5))))
        lipids[d1].append((mean_OD(y,x,930,5)-mean_OD(y,x,850,5)))
        water[d1].append(((mean_OD(y,x,970,5)-mean_OD(y,x,850,5))))

In [ ]:
r = np.array(Rs[::]).astype(float)

In [ ]:
ds = [0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]
min_ratio = []
for i,d1 in enumerate(ds):
    plt.plot(r,ratio[d1],'-',label=f'{d1}')
    min_ratio.append(r[np.argmin(ratio[d1])])
plt.xlabel('distance, cm')
plt.ylabel('lipids/water, a.u.')
plt.legend(title='dermis thickness, cm',loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
ds = [0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]

min_ratio = []
for i,d1 in enumerate(ds):
    plt.plot(r,lipids[d1],'-',label=f'{d1}')
    min_ratio.append(r[np.argmin(ratio[d1])])
plt.xlabel('distance, cm')
plt.ylabel('lipids, a.u.')
plt.legend(title='dermis thickness, cm',loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
ds = [0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]
min_ratio = []
for i,d1 in enumerate(ds):
    plt.plot(r,water[d1],'-',label=f'{d1}')
    min_ratio.append(r[np.argmin(ratio[d1])])
plt.xlabel('distance, cm')
plt.ylabel('water, a.u.')
plt.legend(title='dermis thickness, cm',loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
ds = [0.075, 0.1, 0.125, 0.15, 0.175, 0.2, 0.225, 0.25, 0.275]
min_ratio = []
for i,d1 in enumerate(ds):
    plt.plot(r,ratio[d1],'-',label=f'{d1}')
    plt.axvline(d1)

    min_ratio.append(r[np.argmin(ratio[d1])])
plt.xlabel('distance, cm')
plt.ylabel('lipids/water, a.u.')
plt.legend(title='dermis thickness, cm',loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
plt.plot(ds,min_ratio,'o')
plt.xlabel('Dermis thickness, cm')
plt.ylabel('Min ratio position, cm')